<a href="https://colab.research.google.com/github/PortugalJip2022/Portugal2022/blob/main/%E6%89%8B%E6%9B%B8%E3%81%8D%E6%96%87%E5%AD%97%E3%83%81%E3%83%A5%E3%83%BC%E3%83%88%E3%83%AA%E3%82%A2%E3%83%AB_%E6%B0%B4%E8%B0%B7_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 事前準備

In [ ]:
# リポジトリをコピー(colab上直下に以下フォルダが作成される）

!git clone https://$$TOKEN@github.com/PortugalJip2022/image-recognition.git

# zipファイルを解凍（colab上直下に解凍フォルダが作成される）
!unzip /content/image-recognition/mnist/train_1.zip
!unzip /content/image-recognition/mnist/train_2.zip

!unzip /content/image-recognition/mnist/test.zip

ストリーミング出力は最後の 5000 行に切り捨てられました。
 extracting: test/test_5499.jpg      
 extracting: test/test_55.jpg        
 extracting: test/test_550.jpg       
 extracting: test/test_5500.jpg      
 extracting: test/test_5501.jpg      
 extracting: test/test_5502.jpg      
 extracting: test/test_5503.jpg      
 extracting: test/test_5504.jpg      
 extracting: test/test_5505.jpg      
 extracting: test/test_5506.jpg      
 extracting: test/test_5507.jpg      
 extracting: test/test_5508.jpg      
 extracting: test/test_5509.jpg      
 extracting: test/test_551.jpg       
 extracting: test/test_5510.jpg      
 extracting: test/test_5511.jpg      
 extracting: test/test_5512.jpg      
 extracting: test/test_5513.jpg      
 extracting: test/test_5514.jpg      
 extracting: test/test_5515.jpg      
 extracting: test/test_5516.jpg      
 extracting: test/test_5517.jpg      
 extracting: test/test_5518.jpg      
 extracting: test/test_5519.jpg      
 extracting: test/test_552.jpg       
 extracting: test/

In [ ]:
# train_1とtrain_2を1つのフォルダにまとめる

import os
import shutil

# フォルダ作成
!mkdir train

# train_1からtrainへファイル移動
for filename in os.listdir('/content/train_1'):
  shutil.move(os.path.join('/content/train_1',filename),'/content/train')

# train_2からtrainへファイル移動
for filename in os.listdir('/content/train_2'):
  shutil.move(os.path.join('/content/train_2',filename),'/content/train')

# train_1とtrain_2のフォルダを削除
shutil.rmtree('/content/train_1')
shutil.rmtree('/content/train_2')


# フォルダ内のファイル数確認
import os

path_1 =  "/content/train"
files_1 = os.listdir(path_1)
num_1 = len(files_1)

path_2 =  "/content/test"
files_2 = os.listdir(path_2)
num_2 = len(files_2)

print('trainフォルダ画像数：',num_1)  # 60000になったらOK
print('test フォルダ画像数：',num_2)  # 10000になったらOK

trainフォルダ画像数： 60000
test フォルダ画像数： 10000


# CNN　具体的に何してる？？  
・「事前準備」と「チュートリアル実行」は何も考えずそのまま実行  
・「テスト用」から、具体的に何してる？を実行

# チュートリアル実行

In [ ]:
!pip install chainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chainer: filename=chainer-7.8.1-py3-none-any.whl size=967716 sha256=ec3420eeb331b27a6dac0b3634c0750ad1bbf478952e825220feba218f7b2ce3
  Stored in directory: /root/.cache/pip/wheels/c4/95/6a/16014db6f761c4e742755b64aac60dbe142da1df6c5919f790
Successfully built chainer


In [ ]:
import numpy as np
from chainer import Chain
import chainer.links as L
import chainer.functions as F

In [ ]:
import os

# チュートリアル記載コード
# _train_images_path = os.path.join('..', 'data', 'image','mnist','open','train')
# _test_images_path = os.path.join('..', 'data', 'image', 'mnist','open','test')
# _train_meta_path = os.path.join('..','data','image','mnist','open','train_master.tsv')
_train_images_path = '/content/train'
_test_images_path = '/content/test'
_train_meta_path = '/content/image-recognition/mnist/train_master.tsv'

学習用画像データと数字の対応表を確認してみます.

In [ ]:
import pandas as pd

In [ ]:
train_master = pd.read_csv(_train_meta_path, sep='\t')
train_master.head()

,file_name,category_id
0,train_0.jpg,5
1,train_1.jpg,0
2,train_2.jpg,4
3,train_3.jpg,1
4,train_4.jpg,9


実際に画像を確認してみます.

In [ ]:
from PIL import Image

In [ ]:
image = Image.open(os.path.join(_test_images_path, 'test_0.jpg'))
print('width, height:', image.size)
image

width, height: (28, 28)


In [ ]:
image = Image.open(os.path.join(_train_images_path, 'train_1.jpg'))
print('width, height:', image.size)
image

width, height: (28, 28)


In [ ]:
image = Image.open(os.path.join(_train_images_path, 'train_2.jpg'))
print('width, height:', image.size)
image

width, height: (28, 28)


学習用画像データにおけるそれぞれの数字の分布をみてみます.

In [ ]:
train_master['category_id'].value_counts().sort_index()

category_id
0    5923
1    6742
2    5958
3    6131
4    5842
5    5421
6    5918
7    6265
8    5851
9    5949
Name: count, dtype: int64

# テスト用

## 画像指定

In [ ]:
# 画像1ファイルのみ抽出
data =  train_master[0:1]

In [ ]:
# 中身確認
print(data)

     file_name  category_id
0  train_0.jpg            5


In [ ]:
# 配列を作成して、画像ファイルを数値配列に変更→格納

import numpy as np

num_train = len(train_master)
image_size = (28,28)

## 0で初期化した配列作成
train_X = np.zeros((num_train,)+(1,)+image_size)
train_Y = np.zeros((num_train,))

data =  train_master[0:1]
image =Image.open('/content/train/train_0.jpg')
image_array = np.array(image).reshape((1,1,)+image_size)
train_X= image_array/255
train_Y = data['category_id']

In [ ]:
train_X = train_X.astype(np.float32)
train_Y = train_Y.astype(np.int32)

In [ ]:
print(train_X.shape)
print(train_Y.shape)

(1, 1, 28, 28)
(1,)


## 以下コードでやってること
 LeNetのクラスを定義  
各層で何が出力されてくるかを見たかったので、それぞれについて以下を実行  
１．**returnの値をそれぞれ変えて**、LeNetクラスを定義
（return以外のコードは変えていないはず…）  
２．LeNetクラスのインスタンスを作成  
３．結果表示  


### 畳み込み層１

In [ ]:
from chainer import Chain
import chainer.links as L
import chainer.functions as F

# シード値を設定（算出される値を固定したい）
np.random.seed(3)

class LeNet(Chain):
    def __init__(self, out_size=10, act_func = ''):
        super(LeNet, self).__init__(
                    conv1 = L.Convolution2D(None, 6, 5, stride=1),
                    conv2 = L.Convolution2D(None, 16, 5, stride=1),
                    fc3 = L.Linear(None, 120),
                    fc4 = L.Linear(None, 84),
                    fc5 = L.Linear(None, out_size))
        self.train = True
        if act_func == 'sigmoid':
            print('activation function is', act_func)
            self.act_func = F.sigmoid
        elif act_func == 'relu':
            print('activation function is', act_func)
            self.act_func = F.relu
        else:
            print('activation function is', 'tanh')
            self.act_func = F.tanh

    def __call__(self, x):
        with chainer.using_config('enable_backprop', self.train):
           # h = F.max_pooling_2d(self.act_func(self.conv1(x)), 2, stride=2)
            c1_b = self.conv1(x)                            #畳み込み層１
            c1_f = self.act_func(c1_b)                      #畳み込み層１に活性化関数適用
            c1_p = F.max_pooling_2d(c1_f, 2, stride=2)      #畳み込み層１にプーリング実施

            # h = F.max_pooling_2d(self.act_func(self.conv2(x)), 2, stride=2)
            c2_b = self.conv2(c1_p)                         #畳み込み層２
            c2_f = self.act_func(c2_b)                      #畳み込み層２に活性化関数適用
            c2_p = F.max_pooling_2d(c2_f, 2, stride=2)      #畳み込み層２にプーリング実施

            # h = self.act_func(self.fc3(h))
            f3_b = self.fc3(c2_p)                           #全結合層３
            f3_f = self.act_func(f3_b)                      #全結合層３に活性化関数適用

            # h = self.act_func(self.fc4(h))
            f4_b = self.fc4(f3_f)                           #全結合層４
            f4_f = self.act_func(f4_b)                      #全結合層４に活性化関数適用

            # y = self.fc5(h4)
            f5_b = self.fc5(f4_f)                           #全結合層５

            return c1_b

In [ ]:
import chainer

lenet_c1_b = LeNet()

with chainer.using_config('train',False):
  result_c1_b =lenet_c1_b(train_X).array

# print(result_c1_b)

activation function is tanh


### 畳み込み層１に活性化関数適用

In [ ]:
from chainer import Chain
import chainer.links as L
import chainer.functions as F

# シード値を設定（算出される値を固定したい）
np.random.seed(3)

class LeNet(Chain):
    def __init__(self, out_size=10, act_func = ''):
        super(LeNet, self).__init__(
                    conv1 = L.Convolution2D(None, 6, 5, stride=1),
                    conv2 = L.Convolution2D(None, 16, 5, stride=1),
                    fc3 = L.Linear(None, 120),
                    fc4 = L.Linear(None, 84),
                    fc5 = L.Linear(None, out_size))
        self.train = True
        if act_func == 'sigmoid':
            print('activation function is', act_func)
            self.act_func = F.sigmoid
        elif act_func == 'relu':
            print('activation function is', act_func)
            self.act_func = F.relu
        else:
            print('activation function is', 'tanh')
            self.act_func = F.tanh

    def __call__(self, x):
        with chainer.using_config('enable_backprop', self.train):
           # h = F.max_pooling_2d(self.act_func(self.conv1(x)), 2, stride=2)
            c1_b = self.conv1(x)                            #畳み込み層１
            c1_f = self.act_func(c1_b)                      #畳み込み層１に活性化関数適用
            c1_p = F.max_pooling_2d(c1_f, 2, stride=2)      #畳み込み層１にプーリング実施

            # h = F.max_pooling_2d(self.act_func(self.conv2(x)), 2, stride=2)
            c2_b = self.conv2(c1_p)                         #畳み込み層２
            c2_f = self.act_func(c2_b)                      #畳み込み層２に活性化関数適用
            c2_p = F.max_pooling_2d(c2_f, 2, stride=2)      #畳み込み層２にプーリング実施

            # h = self.act_func(self.fc3(h))
            f3_b = self.fc3(c2_p)                           #全結合層３
            f3_f = self.act_func(f3_b)                      #全結合層３に活性化関数適用

            # h = self.act_func(self.fc4(h))
            f4_b = self.fc4(f3_f)                           #全結合層４
            f4_f = self.act_func(f4_b)                      #全結合層４に活性化関数適用

            # y = self.fc5(h4)
            f5_b = self.fc5(f4_f)                           #全結合層５

            return c1_f

In [ ]:
import chainer

lenet_c1_f = LeNet()

with chainer.using_config('train',False):
  result_c1_f =lenet_c1_f(train_X).array

# print(result_c1_f)

activation function is tanh


### 畳み込み層１にプーリング実施

In [ ]:
from chainer import Chain
import chainer.links as L
import chainer.functions as F

# シード値を設定（算出される値を固定したい）
np.random.seed(3)

class LeNet(Chain):
    def __init__(self, out_size=10, act_func = ''):
        super(LeNet, self).__init__(
                    conv1 = L.Convolution2D(None, 6, 5, stride=1),
                    conv2 = L.Convolution2D(None, 16, 5, stride=1),
                    fc3 = L.Linear(None, 120),
                    fc4 = L.Linear(None, 84),
                    fc5 = L.Linear(None, out_size))
        self.train = True
        if act_func == 'sigmoid':
            print('activation function is', act_func)
            self.act_func = F.sigmoid
        elif act_func == 'relu':
            print('activation function is', act_func)
            self.act_func = F.relu
        else:
            print('activation function is', 'tanh')
            self.act_func = F.tanh

    def __call__(self, x):
        with chainer.using_config('enable_backprop', self.train):
           # h = F.max_pooling_2d(self.act_func(self.conv1(x)), 2, stride=2)
            c1_b = self.conv1(x)                            #畳み込み層１
            c1_f = self.act_func(c1_b)                      #畳み込み層１に活性化関数適用
            c1_p = F.max_pooling_2d(c1_f, 2, stride=2)      #畳み込み層１にプーリング実施

            # h = F.max_pooling_2d(self.act_func(self.conv2(x)), 2, stride=2)
            c2_b = self.conv2(c1_p)                         #畳み込み層２
            c2_f = self.act_func(c2_b)                      #畳み込み層２に活性化関数適用
            c2_p = F.max_pooling_2d(c2_f, 2, stride=2)      #畳み込み層２にプーリング実施

            # h = self.act_func(self.fc3(h))
            f3_b = self.fc3(c2_p)                           #全結合層３
            f3_f = self.act_func(f3_b)                      #全結合層３に活性化関数適用

            # h = self.act_func(self.fc4(h))
            f4_b = self.fc4(f3_f)                           #全結合層４
            f4_f = self.act_func(f4_b)                      #全結合層４に活性化関数適用

            # y = self.fc5(h4)
            f5_b = self.fc5(f4_f)                           #全結合層５

            return c1_p

In [ ]:
import chainer

lenet_c1_p = LeNet()

with chainer.using_config('train',False):
  result_c1_p =lenet_c1_p(train_X).array

# print(result_c1_p)

activation function is tanh


### 畳み込み層２

In [ ]:
from chainer import Chain
import chainer.links as L
import chainer.functions as F

# シード値を設定（算出される値を固定したい）
np.random.seed(3)

class LeNet(Chain):
    def __init__(self, out_size=10, act_func = ''):
        super(LeNet, self).__init__(
                    conv1 = L.Convolution2D(None, 6, 5, stride=1),
                    conv2 = L.Convolution2D(None, 16, 5, stride=1),
                    fc3 = L.Linear(None, 120),
                    fc4 = L.Linear(None, 84),
                    fc5 = L.Linear(None, out_size))
        self.train = True
        if act_func == 'sigmoid':
            print('activation function is', act_func)
            self.act_func = F.sigmoid
        elif act_func == 'relu':
            print('activation function is', act_func)
            self.act_func = F.relu
        else:
            print('activation function is', 'tanh')
            self.act_func = F.tanh

    def __call__(self, x):
        with chainer.using_config('enable_backprop', self.train):
           # h = F.max_pooling_2d(self.act_func(self.conv1(x)), 2, stride=2)
            c1_b = self.conv1(x)                            #畳み込み層１
            c1_f = self.act_func(c1_b)                      #畳み込み層１に活性化関数適用
            c1_p = F.max_pooling_2d(c1_f, 2, stride=2)      #畳み込み層１にプーリング実施

            # h = F.max_pooling_2d(self.act_func(self.conv2(x)), 2, stride=2)
            c2_b = self.conv2(c1_p)                         #畳み込み層２
            c2_f = self.act_func(c2_b)                      #畳み込み層２に活性化関数適用
            c2_p = F.max_pooling_2d(c2_f, 2, stride=2)      #畳み込み層２にプーリング実施

            # h = self.act_func(self.fc3(h))
            f3_b = self.fc3(c2_p)                           #全結合層３
            f3_f = self.act_func(f3_b)                      #全結合層３に活性化関数適用

            # h = self.act_func(self.fc4(h))
            f4_b = self.fc4(f3_f)                           #全結合層４
            f4_f = self.act_func(f4_b)                      #全結合層４に活性化関数適用

            # y = self.fc5(h4)
            f5_b = self.fc5(f4_f)                           #全結合層５

            return c2_b

In [ ]:
import chainer

lenet_c2_b = LeNet()

with chainer.using_config('train',False):
  result_c2_b =lenet_c2_b(train_X).array

# print(result_c2_b)

activation function is tanh


### 畳み込み層２に活性化関数適用

In [ ]:
from chainer import Chain
import chainer.links as L
import chainer.functions as F

# シード値を設定（算出される値を固定したい）
np.random.seed(3)

class LeNet(Chain):
    def __init__(self, out_size=10, act_func = ''):
        super(LeNet, self).__init__(
                    conv1 = L.Convolution2D(None, 6, 5, stride=1),
                    conv2 = L.Convolution2D(None, 16, 5, stride=1),
                    fc3 = L.Linear(None, 120),
                    fc4 = L.Linear(None, 84),
                    fc5 = L.Linear(None, out_size))
        self.train = True
        if act_func == 'sigmoid':
            print('activation function is', act_func)
            self.act_func = F.sigmoid
        elif act_func == 'relu':
            print('activation function is', act_func)
            self.act_func = F.relu
        else:
            print('activation function is', 'tanh')
            self.act_func = F.tanh

    def __call__(self, x):
        with chainer.using_config('enable_backprop', self.train):
           # h = F.max_pooling_2d(self.act_func(self.conv1(x)), 2, stride=2)
            c1_b = self.conv1(x)                            #畳み込み層１
            c1_f = self.act_func(c1_b)                      #畳み込み層１に活性化関数適用
            c1_p = F.max_pooling_2d(c1_f, 2, stride=2)      #畳み込み層１にプーリング実施

            # h = F.max_pooling_2d(self.act_func(self.conv2(x)), 2, stride=2)
            c2_b = self.conv2(c1_p)                         #畳み込み層２
            c2_f = self.act_func(c2_b)                      #畳み込み層２に活性化関数適用
            c2_p = F.max_pooling_2d(c2_f, 2, stride=2)      #畳み込み層２にプーリング実施

            # h = self.act_func(self.fc3(h))
            f3_b = self.fc3(c2_p)                           #全結合層３
            f3_f = self.act_func(f3_b)                      #全結合層３に活性化関数適用

            # h = self.act_func(self.fc4(h))
            f4_b = self.fc4(f3_f)                           #全結合層４
            f4_f = self.act_func(f4_b)                      #全結合層４に活性化関数適用

            # y = self.fc5(h4)
            f5_b = self.fc5(f4_f)                           #全結合層５

            return c2_f

In [ ]:
import chainer

lenet_c2_f = LeNet()

with chainer.using_config('train',False):
  result_c2_f =lenet_c2_f(train_X).array

# print(result_c2_f)

activation function is tanh


### 畳み込み層２にプーリング実施

In [ ]:
from chainer import Chain
import chainer.links as L
import chainer.functions as F

# シード値を設定（算出される値を固定したい）
np.random.seed(3)

class LeNet(Chain):
    def __init__(self, out_size=10, act_func = ''):
        super(LeNet, self).__init__(
                    conv1 = L.Convolution2D(None, 6, 5, stride=1),
                    conv2 = L.Convolution2D(None, 16, 5, stride=1),
                    fc3 = L.Linear(None, 120),
                    fc4 = L.Linear(None, 84),
                    fc5 = L.Linear(None, out_size))
        self.train = True
        if act_func == 'sigmoid':
            print('activation function is', act_func)
            self.act_func = F.sigmoid
        elif act_func == 'relu':
            print('activation function is', act_func)
            self.act_func = F.relu
        else:
            print('activation function is', 'tanh')
            self.act_func = F.tanh

    def __call__(self, x):
        with chainer.using_config('enable_backprop', self.train):
           # h = F.max_pooling_2d(self.act_func(self.conv1(x)), 2, stride=2)
            c1_b = self.conv1(x)                            #畳み込み層１
            c1_f = self.act_func(c1_b)                      #畳み込み層１に活性化関数適用
            c1_p = F.max_pooling_2d(c1_f, 2, stride=2)      #畳み込み層１にプーリング実施

            # h = F.max_pooling_2d(self.act_func(self.conv2(x)), 2, stride=2)
            c2_b = self.conv2(c1_p)                         #畳み込み層２
            c2_f = self.act_func(c2_b)                      #畳み込み層２に活性化関数適用
            c2_p = F.max_pooling_2d(c2_f, 2, stride=2)      #畳み込み層２にプーリング実施

            # h = self.act_func(self.fc3(h))
            f3_b = self.fc3(c2_p)                           #全結合層３
            f3_f = self.act_func(f3_b)                      #全結合層３に活性化関数適用

            # h = self.act_func(self.fc4(h))
            f4_b = self.fc4(f3_f)                           #全結合層４
            f4_f = self.act_func(f4_b)                      #全結合層４に活性化関数適用

            # y = self.fc5(h4)
            f5_b = self.fc5(f4_f)                           #全結合層５

            return c2_p

In [ ]:
import chainer

lenet_c2_p = LeNet()

with chainer.using_config('train',False):
  result_c2_p =lenet_c2_p(train_X).array

# print(result_c2_p)

activation function is tanh


###全結合層３

In [ ]:
from chainer import Chain
import chainer.links as L
import chainer.functions as F

# シード値を設定（算出される値を固定したい）
np.random.seed(3)

class LeNet(Chain):
    def __init__(self, out_size=10, act_func = ''):
        super(LeNet, self).__init__(
                    conv1 = L.Convolution2D(None, 6, 5, stride=1),
                    conv2 = L.Convolution2D(None, 16, 5, stride=1),
                    fc3 = L.Linear(None, 120),
                    fc4 = L.Linear(None, 84),
                    fc5 = L.Linear(None, out_size))
        self.train = True
        if act_func == 'sigmoid':
            print('activation function is', act_func)
            self.act_func = F.sigmoid
        elif act_func == 'relu':
            print('activation function is', act_func)
            self.act_func = F.relu
        else:
            print('activation function is', 'tanh')
            self.act_func = F.tanh

    def __call__(self, x):
        with chainer.using_config('enable_backprop', self.train):
           # h = F.max_pooling_2d(self.act_func(self.conv1(x)), 2, stride=2)
            c1_b = self.conv1(x)                            #畳み込み層１
            c1_f = self.act_func(c1_b)                      #畳み込み層１に活性化関数適用
            c1_p = F.max_pooling_2d(c1_f, 2, stride=2)      #畳み込み層１にプーリング実施

            # h = F.max_pooling_2d(self.act_func(self.conv2(x)), 2, stride=2)
            c2_b = self.conv2(c1_p)                         #畳み込み層２
            c2_f = self.act_func(c2_b)                      #畳み込み層２に活性化関数適用
            c2_p = F.max_pooling_2d(c2_f, 2, stride=2)      #畳み込み層２にプーリング実施

            # h = self.act_func(self.fc3(h))
            f3_b = self.fc3(c2_p)                           #全結合層３
            f3_f = self.act_func(f3_b)                      #全結合層３に活性化関数適用

            # h = self.act_func(self.fc4(h))
            f4_b = self.fc4(f3_f)                           #全結合層４
            f4_f = self.act_func(f4_b)                      #全結合層４に活性化関数適用

            # y = self.fc5(h4)
            f5_b = self.fc5(f4_f)                           #全結合層５

            return f3_b

In [ ]:
import chainer

lenet_f3_b = LeNet()

with chainer.using_config('train',False):
  result_f3_b =lenet_f3_b(train_X).array

# print(result_f3_b)

activation function is tanh


###全結合層３に活性化関数適用

In [ ]:
from chainer import Chain
import chainer.links as L
import chainer.functions as F

# シード値を設定（算出される値を固定したい）
np.random.seed(3)

class LeNet(Chain):
    def __init__(self, out_size=10, act_func = ''):
        super(LeNet, self).__init__(
                    conv1 = L.Convolution2D(None, 6, 5, stride=1),
                    conv2 = L.Convolution2D(None, 16, 5, stride=1),
                    fc3 = L.Linear(None, 120),
                    fc4 = L.Linear(None, 84),
                    fc5 = L.Linear(None, out_size))
        self.train = True
        if act_func == 'sigmoid':
            print('activation function is', act_func)
            self.act_func = F.sigmoid
        elif act_func == 'relu':
            print('activation function is', act_func)
            self.act_func = F.relu
        else:
            print('activation function is', 'tanh')
            self.act_func = F.tanh

    def __call__(self, x):
        with chainer.using_config('enable_backprop', self.train):
           # h = F.max_pooling_2d(self.act_func(self.conv1(x)), 2, stride=2)
            c1_b = self.conv1(x)                            #畳み込み層１
            c1_f = self.act_func(c1_b)                      #畳み込み層１に活性化関数適用
            c1_p = F.max_pooling_2d(c1_f, 2, stride=2)      #畳み込み層１にプーリング実施

            # h = F.max_pooling_2d(self.act_func(self.conv2(x)), 2, stride=2)
            c2_b = self.conv2(c1_p)                         #畳み込み層２
            c2_f = self.act_func(c2_b)                      #畳み込み層２に活性化関数適用
            c2_p = F.max_pooling_2d(c2_f, 2, stride=2)      #畳み込み層２にプーリング実施

            # h = self.act_func(self.fc3(h))
            f3_b = self.fc3(c2_p)                           #全結合層３
            f3_f = self.act_func(f3_b)                      #全結合層３に活性化関数適用

            # h = self.act_func(self.fc4(h))
            f4_b = self.fc4(f3_f)                           #全結合層４
            f4_f = self.act_func(f4_b)                      #全結合層４に活性化関数適用

            # y = self.fc5(h4)
            f5_b = self.fc5(f4_f)                           #全結合層５

            return f3_f

In [ ]:
import chainer

lenet_f3_f = LeNet()

with chainer.using_config('train',False):
  result_f3_f =lenet_f3_f(train_X).array

# print(result_f3_f)

activation function is tanh


###全結合層４

In [ ]:
from chainer import Chain
import chainer.links as L
import chainer.functions as F

# シード値を設定（算出される値を固定したい）
np.random.seed(3)

class LeNet(Chain):
    def __init__(self, out_size=10, act_func = ''):
        super(LeNet, self).__init__(
                    conv1 = L.Convolution2D(None, 6, 5, stride=1),
                    conv2 = L.Convolution2D(None, 16, 5, stride=1),
                    fc3 = L.Linear(None, 120),
                    fc4 = L.Linear(None, 84),
                    fc5 = L.Linear(None, out_size))
        self.train = True
        if act_func == 'sigmoid':
            print('activation function is', act_func)
            self.act_func = F.sigmoid
        elif act_func == 'relu':
            print('activation function is', act_func)
            self.act_func = F.relu
        else:
            print('activation function is', 'tanh')
            self.act_func = F.tanh

    def __call__(self, x):
        with chainer.using_config('enable_backprop', self.train):
           # h = F.max_pooling_2d(self.act_func(self.conv1(x)), 2, stride=2)
            c1_b = self.conv1(x)                            #畳み込み層１
            c1_f = self.act_func(c1_b)                      #畳み込み層１に活性化関数適用
            c1_p = F.max_pooling_2d(c1_f, 2, stride=2)      #畳み込み層１にプーリング実施

            # h = F.max_pooling_2d(self.act_func(self.conv2(x)), 2, stride=2)
            c2_b = self.conv2(c1_p)                         #畳み込み層２
            c2_f = self.act_func(c2_b)                      #畳み込み層２に活性化関数適用
            c2_p = F.max_pooling_2d(c2_f, 2, stride=2)      #畳み込み層２にプーリング実施

            # h = self.act_func(self.fc3(h))
            f3_b = self.fc3(c2_p)                           #全結合層３
            f3_f = self.act_func(f3_b)                      #全結合層３に活性化関数適用

            # h = self.act_func(self.fc4(h))
            f4_b = self.fc4(f3_f)                           #全結合層４
            f4_f = self.act_func(f4_b)                      #全結合層４に活性化関数適用

            # y = self.fc5(h4)
            f5_b = self.fc5(f4_f)                           #全結合層５

            return f4_b

In [ ]:
import chainer

lenet_f4_b = LeNet()

with chainer.using_config('train',False):
  result_f4_b =lenet_f4_b(train_X).array

# print(result_f4_b)

activation function is tanh


###全結合層４に活性化関数適用

In [ ]:
from chainer import Chain
import chainer.links as L
import chainer.functions as F

# シード値を設定（算出される値を固定したい）
np.random.seed(3)

class LeNet(Chain):
    def __init__(self, out_size=10, act_func = ''):
        super(LeNet, self).__init__(
                    conv1 = L.Convolution2D(None, 6, 5, stride=1),
                    conv2 = L.Convolution2D(None, 16, 5, stride=1),
                    fc3 = L.Linear(None, 120),
                    fc4 = L.Linear(None, 84),
                    fc5 = L.Linear(None, out_size))
        self.train = True
        if act_func == 'sigmoid':
            print('activation function is', act_func)
            self.act_func = F.sigmoid
        elif act_func == 'relu':
            print('activation function is', act_func)
            self.act_func = F.relu
        else:
            print('activation function is', 'tanh')
            self.act_func = F.tanh

    def __call__(self, x):
        with chainer.using_config('enable_backprop', self.train):
           # h = F.max_pooling_2d(self.act_func(self.conv1(x)), 2, stride=2)
            c1_b = self.conv1(x)                            #畳み込み層１
            c1_f = self.act_func(c1_b)                      #畳み込み層１に活性化関数適用
            c1_p = F.max_pooling_2d(c1_f, 2, stride=2)      #畳み込み層１にプーリング実施

            # h = F.max_pooling_2d(self.act_func(self.conv2(x)), 2, stride=2)
            c2_b = self.conv2(c1_p)                         #畳み込み層２
            c2_f = self.act_func(c2_b)                      #畳み込み層２に活性化関数適用
            c2_p = F.max_pooling_2d(c2_f, 2, stride=2)      #畳み込み層２にプーリング実施

            # h = self.act_func(self.fc3(h))
            f3_b = self.fc3(c2_p)                           #全結合層３
            f3_f = self.act_func(f3_b)                      #全結合層３に活性化関数適用

            # h = self.act_func(self.fc4(h))
            f4_b = self.fc4(f3_f)                           #全結合層４
            f4_f = self.act_func(f4_b)                      #全結合層４に活性化関数適用

            # y = self.fc5(h4)
            f5_b = self.fc5(f4_f)                           #全結合層５

            return f4_f

In [ ]:
import chainer

lenet_f4_f = LeNet()

with chainer.using_config('train',False):
  result_f4_f =lenet_f4_f(train_X).array

# print(result_f4_f)

activation function is tanh


###全結合層５

In [ ]:
from chainer import Chain
import chainer.links as L
import chainer.functions as F

# シード値を設定（算出される値を固定したい）
np.random.seed(3)

class LeNet(Chain):
    def __init__(self, out_size=10, act_func = ''):
        super(LeNet, self).__init__(
                    conv1 = L.Convolution2D(None, 6, 5, stride=1),
                    conv2 = L.Convolution2D(None, 16, 5, stride=1),
                    fc3 = L.Linear(None, 120),
                    fc4 = L.Linear(None, 84),
                    fc5 = L.Linear(None, out_size))
        self.train = True
        if act_func == 'sigmoid':
            print('activation function is', act_func)
            self.act_func = F.sigmoid
        elif act_func == 'relu':
            print('activation function is', act_func)
            self.act_func = F.relu
        else:
            print('activation function is', 'tanh')
            self.act_func = F.tanh

    def __call__(self, x):
        with chainer.using_config('enable_backprop', self.train):
           # h = F.max_pooling_2d(self.act_func(self.conv1(x)), 2, stride=2)
            c1_b = self.conv1(x)                            #畳み込み層１
            c1_f = self.act_func(c1_b)                      #畳み込み層１に活性化関数適用
            c1_p = F.max_pooling_2d(c1_f, 2, stride=2)      #畳み込み層１にプーリング実施

            # h = F.max_pooling_2d(self.act_func(self.conv2(x)), 2, stride=2)
            c2_b = self.conv2(c1_p)                         #畳み込み層２
            c2_f = self.act_func(c2_b)                      #畳み込み層２に活性化関数適用
            c2_p = F.max_pooling_2d(c2_f, 2, stride=2)      #畳み込み層２にプーリング実施

            # h = self.act_func(self.fc3(h))
            f3_b = self.fc3(c2_p)                           #全結合層３
            f3_f = self.act_func(f3_b)                      #全結合層３に活性化関数適用

            # h = self.act_func(self.fc4(h))
            f4_b = self.fc4(f3_f)                           #全結合層４
            f4_f = self.act_func(f4_b)                      #全結合層４に活性化関数適用

            # y = self.fc5(h4)
            f5_b = self.fc5(f4_f)                           #全結合層５

            return f5_b

In [ ]:
import chainer

lenet_f5_b = LeNet()

# with chainer.using_config('train',False):
with chainer.using_config('enable_backprop',False):
  result_f5_b =lenet_f5_b(train_X).array

# print(result_f5_b)

activation function is tanh


# 結果確認

In [ ]:
# 今回みる画像
image

In [ ]:
# 画像を配列に変換
image_array

array([[[[  0,   0,   1,   1,   3,   5,   3,   0,   0,   0,   0,   1,
            1,   0,   0,   1,   0,   0,   4,   8,   0,  17,   0,   0,
            0,   0,   0,   0],
         [  0,   2,   2,   0,   0,   0,   0,   0,   0,   0,   0,   1,
            1,   1,   1,   3,   0,   0,   1,   0,   0,   0,   9,   5,
            0,   0,   0,   0],
         [  1,   2,   3,   0,   0,   0,   0,   2,   6,   5,   3,   1,
            0,   0,   0,   0,   7,   2,   6,  10,  17,   0,   1,   0,
            0,   0,   0,   0],
         [  3,   0,   0,   2,   2,   0,   1,   4,   1,   3,   2,   0,
            0,   2,   3,   1,   0,   0,   0,   0,   0,   0,   0,  12,
            0,   0,   0,   0],
         [  5,   0,   0,   2,   5,   1,   0,   0,   0,   2,   2,   0,
            0,   1,   0,   0,   5,   0,  18,   0,  16,   4,   0,  11,
            0,   0,   0,   0],
         [  6,   0,   0,   1,   2,   0,   0,   0,   0,   0,   5,   6,
           12,  22,  24,  19, 113, 158, 172,  25, 151, 255, 255, 120,
     

In [ ]:
# 配列に変換後、正規化
train_X

array([[[[0.        , 0.        , 0.00392157, 0.00392157, 0.01176471,
          0.01960784, 0.01176471, 0.        , 0.        , 0.        ,
          0.        , 0.00392157, 0.00392157, 0.        , 0.        ,
          0.00392157, 0.        , 0.        , 0.01568628, 0.03137255,
          0.        , 0.06666667, 0.        , 0.        , 0.        ,
          0.        , 0.        , 0.        ],
         [0.        , 0.00784314, 0.00784314, 0.        , 0.        ,
          0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        , 0.00392157, 0.00392157, 0.00392157, 0.00392157,
          0.01176471, 0.        , 0.        , 0.00392157, 0.        ,
          0.        , 0.        , 0.03529412, 0.01960784, 0.        ,
          0.        , 0.        , 0.        ],
         [0.00392157, 0.00784314, 0.01176471, 0.        , 0.        ,
          0.        , 0.        , 0.00784314, 0.02352941, 0.01960784,
          0.01176471, 0.00392157, 0.        , 0.        , 0.      

##  畳み込み層１
画像をフィルターに通して、特徴を抽出する  
　　　↓  
活性化関数適用  
　　　↓  
プーリングを実行してダウンサンプリング

In [ ]:
# 畳み込み層１のフィルターの値を取得
conv1_param = lenet_c1_b.conv1.W.data
print(conv1_param)

[[[[ 0.35772568  0.08730197  0.01929949 -0.37269855 -0.05547764]
   [-0.0709518  -0.0165483  -0.12540014 -0.00876363 -0.09544361]
   [-0.26277295  0.17692448  0.17626362  0.34191462  0.01000673]
   [-0.08093549 -0.10907199 -0.30929548  0.1964735  -0.22021353]
   [-0.2370093  -0.04112998  0.29722968  0.04734325 -0.20475703]]]


 [[[-0.14259864  0.125049   -0.03210267 -0.15376727 -0.04600614]
   [ 0.14901125  0.39522216 -0.24882467 -0.12528338 -0.16075322]
   [-0.48381662 -0.18475841 -0.20477515  0.2247956  -0.02638285]
   [-0.32465708  0.12933509 -0.07125415 -0.3486282  -0.11932993]
   [-0.11771888 -0.17477646  0.00594276 -0.44965154 -0.05355237]]]


 [[[ 0.20263669  0.17055957  0.2216375   0.22387813  0.29750863]
   [-0.22366014  0.16916668 -0.3721779  -0.12057702 -0.3828944 ]
   [ 0.2096295   0.26674756 -0.03948294  0.354929   -0.1349455 ]
   [ 0.03012337  0.03058914 -0.21283905  0.08758932  0.3877957 ]
   [-0.20498617  0.17986768 -0.03090137  0.35392547  0.09675767]]]


 [[[ 0.135243

In [ ]:
# 畳み込み１後
result_c1_b

array([[[[-1.01501718e-02, -4.14855033e-03, -8.08995496e-03, ...,
           1.60481147e-02, -1.07661746e-02, -1.54238716e-02],
         [-1.27393985e-02,  8.90330318e-03,  8.00629205e-04, ...,
          -1.41188711e-01, -2.38684610e-01, -1.20376684e-01],
         [-6.89000636e-03,  1.35656781e-02,  4.92642447e-03, ...,
          -5.23244739e-01, -3.07814389e-01, -1.10387288e-01],
         ...,
         [-6.98096380e-02, -6.32069334e-02,  7.16619417e-02, ...,
           2.10080128e-02,  8.13156459e-03, -1.11296948e-03],
         [-2.25797128e-02,  9.49627385e-02,  3.15100402e-01, ...,
           7.74638308e-03, -2.80419225e-03,  5.61138336e-03],
         [-6.45706952e-02, -2.34137833e-01, -4.60457951e-01, ...,
           5.13541000e-03,  2.10426878e-02,  0.00000000e+00]],

        [[-2.02521533e-02, -1.63196754e-02, -1.15560722e-02, ...,
          -6.10436965e-03,  9.65833571e-03, -1.74342599e-02],
         [-1.90431830e-02, -5.52827073e-03, -4.91268001e-03, ...,
          -3.02167654e

In [ ]:
# 畳み込み層１に活性化関数適用
result_c1_f

array([[[[-1.01498235e-02, -4.14852658e-03, -8.08977894e-03, ...,
           1.60467383e-02, -1.07657593e-02, -1.54226488e-02],
         [-1.27387093e-02,  8.90306849e-03,  8.00629030e-04, ...,
          -1.40257969e-01, -2.34252915e-01, -1.19798593e-01],
         [-6.88989740e-03,  1.35648465e-02,  4.92638443e-03, ...,
          -4.80200440e-01, -2.98447460e-01, -1.09941088e-01],
         ...,
         [-6.96964562e-02, -6.31228983e-02,  7.15395212e-02, ...,
           2.10049227e-02,  8.13138485e-03, -1.11296901e-03],
         [-2.25758757e-02,  9.46783051e-02,  3.05069983e-01, ...,
           7.74622802e-03, -2.80418480e-03,  5.61132468e-03],
         [-6.44811094e-02, -2.29951084e-01, -4.30457383e-01, ...,
           5.13536483e-03,  2.10395828e-02,  0.00000000e+00]],

        [[-2.02493854e-02, -1.63182262e-02, -1.15555581e-02, ...,
          -6.10429375e-03,  9.65803489e-03, -1.74324941e-02],
         [-1.90408807e-02, -5.52821439e-03, -4.91264043e-03, ...,
          -2.93295056e

In [ ]:
# 畳み込み層１にプーリング実施
result_c1_p

array([[[[ 8.90306849e-03,  8.00629030e-04,  8.02978128e-03,
           1.26732364e-02, -3.40483524e-03,  1.09898252e-02,
           4.58953390e-03,  1.66630611e-01,  2.68690065e-02,
           1.60343513e-01,  1.42973512e-01, -1.07657593e-02],
         [ 1.35648465e-02,  4.92638443e-03, -2.94060092e-02,
           1.84901301e-02, -4.71868739e-02, -7.93704018e-02,
          -1.90517813e-01, -1.09260112e-01, -3.75890046e-01,
           1.96970761e-01, -5.33460714e-02, -1.09941088e-01],
         [ 1.16709983e-02,  2.07646587e-03,  1.60419866e-01,
           2.13815510e-01, -2.07141444e-01, -9.81800184e-02,
          -2.76725471e-01,  2.11043879e-02, -2.66044199e-01,
           1.42309606e-01,  3.07787895e-01,  2.83852130e-01],
         [-2.73271056e-04,  1.03369362e-04,  4.78229932e-02,
           1.13117322e-01,  1.31199300e-01,  2.43781745e-01,
          -2.20179468e-01,  2.50443518e-01,  2.51519799e-01,
           9.98299345e-02,  3.53419065e-01,  2.62535423e-01],
         [-2.1572804

##  畳み込み層２


In [ ]:
# 畳み込み層２のフィルターの値を取得
conv2_param = lenet_c1_b.conv2.W.data
print(conv2_param)

[[[[-7.04071894e-02  8.75026286e-02 -9.96862277e-02  4.86757979e-03
     1.99564995e-04]
   [ 3.46713625e-02 -5.92313968e-02 -2.85311555e-03 -1.14815775e-02
     8.14119279e-02]
   [-6.49861619e-02  5.93964756e-03 -2.13301964e-02 -1.05985068e-01
     2.18503669e-01]
   [-5.81500912e-03 -1.21385120e-01  1.15013912e-01 -8.74129385e-02
     3.02814059e-02]
   [ 7.04499483e-02 -5.29442541e-02 -3.51820253e-02 -4.41128835e-02
    -1.05622821e-02]]

  [[-1.32473394e-01 -1.00889310e-01 -1.14951646e-02  8.48300830e-02
     5.15816957e-02]
   [ 1.41206339e-01  5.66691309e-02 -4.17335071e-02 -1.00301225e-02
    -1.65780947e-01]
   [-7.84469619e-02 -8.33119899e-02  2.20938604e-02  5.28950803e-02
    -4.57542986e-02]
   [-4.80509549e-02 -1.26275748e-01 -1.04317283e-02  2.02628337e-02
     3.63978632e-02]
   [-6.39079288e-02  1.62359506e-01  9.75761041e-02 -7.77611742e-03
    -4.30447012e-02]]

  [[-2.62572803e-02  1.23397429e-02 -1.52094662e-03  3.94799598e-02
     6.27857447e-02]
   [ 1.11553259e-

In [ ]:
#畳み込み層２実施
result_c2_b

array([[[[ 0.36370158,  0.68130046,  0.7420022 , ...,  0.9158568 ,
           0.6925121 ,  0.47942808],
         [ 0.2413388 ,  0.39931366,  0.50895673, ...,  0.5388336 ,
           0.495736  ,  0.14113139],
         [ 0.19258305,  0.5157807 ,  0.53010595, ...,  0.60153717,
           0.5135061 ,  0.1962746 ],
         ...,
         [ 0.2225973 ,  0.08134315, -0.0384531 , ...,  0.3260627 ,
           0.612241  ,  0.6340414 ],
         [-0.04729144,  0.04259328,  0.29123944, ...,  0.9744519 ,
           0.9125798 ,  0.38496056],
         [ 0.38843858,  0.60650015,  0.5349707 , ...,  0.43557623,
           0.6299126 ,  0.41714558]],

        [[-0.34363186, -0.24621347,  0.06433733, ...,  0.7607323 ,
           0.5127211 ,  0.25273252],
         [-0.0799017 , -0.12360006,  0.03203174, ...,  0.6825424 ,
           0.42345366,  0.2844266 ],
         [ 0.29902953, -0.06874385, -0.289788  , ...,  0.687844  ,
           0.46343178,  0.4062244 ],
         ...,
         [ 0.26225865,  0.17509325

In [ ]:
# 畳み込み層２実施後に活性化関数適用
result_c2_f

array([[[[ 0.34847033,  0.5923642 ,  0.63035333, ...,  0.72393143,
           0.5995934 ,  0.44578546],
         [ 0.2367599 ,  0.37936157,  0.46913195, ...,  0.4921046 ,
           0.45875713,  0.14020176],
         [ 0.190237  ,  0.474437  ,  0.4854621 , ...,  0.5381425 ,
           0.47267252,  0.19379243],
         ...,
         [ 0.2189922 ,  0.08116422, -0.03843416, ...,  0.31497854,
           0.5457027 ,  0.5608288 ],
         [-0.04725622,  0.04256754,  0.2832752 , ...,  0.7506541 ,
           0.7223681 ,  0.36700767],
         [ 0.3700134 ,  0.5416588 ,  0.48917156, ...,  0.4099709 ,
           0.55799204,  0.39452302]],

        [[-0.33071592, -0.24135597,  0.0642487 , ...,  0.6415081 ,
           0.47206268,  0.24748555],
         [-0.0797321 , -0.12297447,  0.03202078, ...,  0.59316975,
           0.39983597,  0.27699703],
         [ 0.29042426, -0.06863577, -0.2819397 , ...,  0.59659517,
           0.43287706,  0.3852622 ],
         ...,
         [ 0.2564069 ,  0.17332561

In [ ]:
# 畳み込み層２に活性化関数適用後、プーリング実装
result_c2_p

array([[[[ 5.9236419e-01,  7.2940493e-01,  7.2393143e-01,
           5.9959340e-01],
         [ 4.7443700e-01,  6.8908906e-01,  5.7906246e-01,
           4.7267252e-01],
         [ 4.6093869e-01,  4.4037083e-01,  5.4093540e-01,
           5.6082880e-01],
         [ 5.4165882e-01,  5.1088655e-01,  7.7230740e-01,
           7.2236812e-01]],

        [[-7.9732098e-02,  3.1087673e-01,  6.4150810e-01,
           4.7206268e-01],
         [ 2.9042426e-01,  2.4938057e-03,  6.7315358e-01,
           5.6306970e-01],
         [ 2.5640690e-01,  5.6872338e-01,  3.6428291e-01,
           5.5890751e-01],
         [ 3.2677001e-01,  4.9641928e-01,  5.8056802e-01,
           3.4490499e-01]],

        [[ 1.9482723e-01, -2.5823677e-03,  1.1258263e-01,
          -2.9717798e-03],
         [ 3.0317628e-01,  1.0918457e-01,  3.6774207e-02,
           1.2258055e-01],
         [ 2.7607709e-01,  2.6865101e-01, -2.2912897e-01,
           1.4352901e-01],
         [ 2.3522104e-01, -9.6252806e-02, -1.7284493e-01,
   

##  全結合層
画像をフィルターに通して、特徴を抽出する


In [ ]:
# 全結合層３適用
result_f3_b

array([[-0.11841824,  0.12413573, -0.7327121 , -0.0766784 , -0.28022045,
        -0.7631141 , -0.4265145 , -0.42986453, -0.49313015,  0.18111277,
        -0.3113046 , -0.26944822,  0.11774178, -0.21433072,  0.0817202 ,
         0.01706874,  0.48212114,  0.13500175,  0.36638495,  0.5631454 ,
        -0.71969646,  0.02166072, -0.06697031,  0.6878482 ,  0.06480342,
        -0.46253166, -0.6210935 , -0.11814918, -0.09267758, -1.1587496 ,
        -0.29255626, -0.07809073, -0.18060465,  0.4439042 , -0.17348395,
         0.27848163, -0.65467775,  0.3653211 ,  0.15392694,  0.1666226 ,
         0.08944474,  0.73643136,  0.53874546,  0.6831459 ,  0.01411144,
        -0.11056022,  0.69523335,  0.44611987,  0.07772309,  0.24205542,
        -0.7751996 , -0.658366  , -0.5707408 , -0.45794702, -0.39657816,
        -0.6863048 , -0.55463254, -0.13705939, -0.04399385,  0.79379034,
         0.06783703, -0.51770747,  0.06722143,  0.71744794, -0.25753862,
         0.70999587,  0.2258731 ,  0.03390104, -0.3

In [ ]:
# 全結合層３適用後、活性化関数適用
result_f3_f

array([[-0.11786781,  0.12350201, -0.62472177, -0.07652847, -0.2731091 ,
        -0.6429076 , -0.40240434, -0.40520808, -0.45669726,  0.17915814,
        -0.30162346, -0.26311132,  0.11720069, -0.21110797,  0.08153877,
         0.01706709,  0.44794074,  0.13418752,  0.35082564,  0.5103075 ,
        -0.6167213 ,  0.02165733, -0.06687037,  0.59659785,  0.06471286,
        -0.43214533, -0.55188894, -0.11760247, -0.09241315, -0.820632  ,
        -0.28448588, -0.07793238, -0.1786663 ,  0.41687536, -0.17176422,
         0.27149922, -0.57481056,  0.34989238,  0.15272266,  0.16509755,
         0.08920697,  0.62698424,  0.49203777,  0.59356076,  0.01411051,
        -0.11011193,  0.6013335 ,  0.4187043 ,  0.07756696,  0.23743622,
        -0.6499428 , -0.577275  , -0.5159032 , -0.4284095 , -0.37701732,
        -0.5956029 , -0.50398415, -0.13620755, -0.04396548,  0.66055095,
         0.06773316, -0.47592872,  0.06712037,  0.615326  , -0.25199187,
         0.61067426,  0.22210865,  0.03388806, -0.3

In [ ]:
# 全結合層４適用
result_f4_b

array([[-2.53450304e-01, -4.18239653e-01, -3.10470402e-01,
        -3.17944765e-01,  3.13775912e-02, -9.76134390e-02,
        -6.57458484e-01, -5.10378838e-01,  6.80726588e-01,
        -3.03379744e-02, -1.59661993e-02,  6.98698223e-01,
        -3.94231752e-02,  2.46704206e-01,  4.05388176e-01,
         1.43239468e-01, -8.21740091e-01, -2.77840018e-01,
         2.34773159e-02,  5.15425086e-01,  8.20461661e-03,
        -7.22381920e-02, -5.44030607e-01, -1.80098280e-01,
         2.41294086e-01,  2.41898835e-01,  6.29953295e-02,
        -2.97152251e-01,  3.13007653e-01,  6.25004053e-01,
         7.86899924e-02, -2.31314972e-01, -1.19345188e-02,
        -3.98611188e-01,  4.39256489e-01,  1.98712163e-02,
         1.27809912e-01,  6.11841440e-01,  4.98913378e-02,
        -5.56094348e-01, -1.49379700e-01, -1.29558802e-01,
         3.54315490e-01,  4.02260035e-01,  5.23567498e-01,
        -3.26164722e-01,  7.14984715e-01,  3.81257474e-01,
         3.08721781e-01,  4.59064513e-01,  2.87373364e-0

In [ ]:
# 全結合層４適用後、活性化関数適用
result_f4_f

array([[-2.48159245e-01, -3.95446420e-01, -3.00864995e-01,
        -3.07647407e-01,  3.13672982e-02, -9.73045826e-02,
        -5.76669574e-01, -4.70240355e-01,  5.91991544e-01,
        -3.03286705e-02, -1.59648433e-02,  6.03540838e-01,
        -3.94027643e-02,  2.41818070e-01,  3.84549856e-01,
         1.42267793e-01, -6.76015913e-01, -2.70904779e-01,
         2.34730039e-02,  4.74161386e-01,  8.20443314e-03,
        -7.21127987e-02, -4.96032953e-01, -1.78176031e-01,
         2.36717686e-01,  2.37288460e-01,  6.29121363e-02,
        -2.88704365e-01,  3.03170800e-01,  5.54602504e-01,
         7.85279721e-02, -2.27275759e-01, -1.19339526e-02,
        -3.78760040e-01,  4.13027942e-01,  1.98686011e-02,
         1.27118483e-01,  5.45422077e-01,  4.98499833e-02,
        -5.05073905e-01, -1.48278430e-01, -1.28838733e-01,
         3.40197176e-01,  3.81881088e-01,  4.80448753e-01,
        -3.15070421e-01,  6.13793135e-01,  3.63799036e-01,
         2.99273819e-01,  4.29321438e-01,  2.79715478e-0

In [ ]:
# 全結合層５
result_f5_b

array([[ 0.75922954,  0.24937296,  0.39605224,  0.7561294 , -1.144773  ,
         0.33356044, -0.15276298, -0.302677  ,  0.2132766 ,  0.00884873]],
      dtype=float32)

##  全結合層_（具体的作業）
画像をフィルターに通して、特徴を抽出する


In [ ]:
# 重み行列を取得
fc3_param = lenet_f3_b.fc3.W.array
fc3_param

array([[-0.10998389,  0.01890443, -0.14056483, ...,  0.03191006,
        -0.07066448, -0.0562632 ],
       [-0.0933558 , -0.07609827,  0.03450594, ...,  0.0459023 ,
         0.02369725,  0.00934355],
       [-0.08900501,  0.09135668,  0.07288211, ...,  0.05576422,
         0.05392596, -0.06035112],
       ...,
       [ 0.00143112, -0.02391433, -0.04665549, ..., -0.09615854,
        -0.02860799,  0.03966358],
       [-0.00375559, -0.09385591,  0.00350007, ..., -0.03089405,
         0.14441688, -0.00863796],
       [-0.02390633, -0.08560088,  0.01598963, ...,  0.02259114,
        -0.01573039, -0.02768728]], dtype=float32)

In [ ]:
# バイアスベクトルを取得
fc3_param_b = lenet_f3_b.fc3.b.array
fc3_param_b

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.], dtype=float32)

In [ ]:
# Excel作業用
fc3_param_new = fc3_param
np.savetxt('weight_param.csv', fc3_param_new)

In [ ]:
# 入力ベクトル作成（16×4×4配列を1次元ベクトルに変換）
input_array = np.array(result_c2_p).reshape(16,4,4)
input_array_flat = input_array.flatten()

In [ ]:
input_array_flat

array([ 5.9236419e-01,  7.2940493e-01,  7.2393143e-01,  5.9959340e-01,
        4.7443700e-01,  6.8908906e-01,  5.7906246e-01,  4.7267252e-01,
        4.6093869e-01,  4.4037083e-01,  5.4093540e-01,  5.6082880e-01,
        5.4165882e-01,  5.1088655e-01,  7.7230740e-01,  7.2236812e-01,
       -7.9732098e-02,  3.1087673e-01,  6.4150810e-01,  4.7206268e-01,
        2.9042426e-01,  2.4938057e-03,  6.7315358e-01,  5.6306970e-01,
        2.5640690e-01,  5.6872338e-01,  3.6428291e-01,  5.5890751e-01,
        3.2677001e-01,  4.9641928e-01,  5.8056802e-01,  3.4490499e-01,
        1.9482723e-01, -2.5823677e-03,  1.1258263e-01, -2.9717798e-03,
        3.0317628e-01,  1.0918457e-01,  3.6774207e-02,  1.2258055e-01,
        2.7607709e-01,  2.6865101e-01, -2.2912897e-01,  1.4352901e-01,
        2.3522104e-01, -9.6252806e-02, -1.7284493e-01,  1.2064531e-01,
        6.3951480e-01,  7.6195443e-01,  6.8114889e-01,  4.6233842e-01,
        5.3609949e-01,  7.1273142e-01,  5.8932996e-01,  3.0715948e-01,
      

In [ ]:
# 入力kベクトルと重み行列の行列積
output = np.dot(fc3_param,input_array_flat)

# バイアスを加算
output += fc3_param_b

In [ ]:
# 結果表示（result_fc3_b)
output

array([-0.11841824,  0.12413573, -0.7327121 , -0.0766784 , -0.28022045,
       -0.7631141 , -0.4265145 , -0.42986453, -0.49313015,  0.18111277,
       -0.3113046 , -0.26944822,  0.11774178, -0.21433072,  0.0817202 ,
        0.01706874,  0.48212114,  0.13500175,  0.36638495,  0.5631454 ,
       -0.71969646,  0.02166072, -0.06697031,  0.6878482 ,  0.06480342,
       -0.46253166, -0.6210935 , -0.11814918, -0.09267758, -1.1587496 ,
       -0.29255626, -0.07809073, -0.18060465,  0.4439042 , -0.17348395,
        0.27848163, -0.65467775,  0.3653211 ,  0.15392694,  0.1666226 ,
        0.08944474,  0.73643136,  0.53874546,  0.6831459 ,  0.01411144,
       -0.11056022,  0.69523335,  0.44611987,  0.07772309,  0.24205542,
       -0.7751996 , -0.658366  , -0.5707408 , -0.45794702, -0.39657816,
       -0.6863048 , -0.55463254, -0.13705939, -0.04399385,  0.79379034,
        0.06783703, -0.51770747,  0.06722143,  0.71744794, -0.25753862,
        0.70999587,  0.2258731 ,  0.03390104, -0.31742826,  0.47

# 元コード

In [ ]:


from chainer import Chain
import chainer.links as L
import chainer.functions as F

class LeNet(Chain):
    def __init__(self, out_size=10, act_func = ''):
        super(LeNet, self).__init__(
                    conv1 = L.Convolution2D(None, 6, 5, stride=1),
                    conv2 = L.Convolution2D(None, 16, 5, stride=1),
                    fc3 = L.Linear(None, 120),
                    fc4 = L.Linear(None, 84),
                    fc5 = L.Linear(None, out_size))
        self.train = True
        if act_func == 'sigmoid':
            print('activation function is', act_func)
            self.act_func = F.sigmoid
        elif act_func == 'relu':
            print('activation function is', act_func)
            self.act_func = F.relu
        else:
            print('activation function is', 'tanh')
            self.act_func = F.tanh

    def __call__(self, x):
        with chainer.using_config('enable_backprop', self.train):
            c1 = self.conv1(x) #畳み込み層１
            # c1 = self.act_func(self.conv1(x)) #畳み込み層１
            # y = F.max_pooling_2d(self.act_func(self.conv1(x)), 2, stride=2)
            # h = F.max_pooling_2d(self.act_func(self.conv2(x)), 2, stride=2)
            # h = self.act_func(self.fc3(h))
            # h = self.act_func(self.fc4(h))
            # y = self.fc5(h4)

            return c1

In [ ]:
lenet1 = LeNet()

activation function is tanh


In [ ]:
# cpuに変更
import chainer
import chainer.functions as F
import chainer.cuda as cuda
from chainer import Chain, optimizers, serializers
#from time import clock
from time import perf_counter

def train_model(model, train, model_name, val = False, batchsize = 128, init_lr = 0.01, num_epochs = 50, cpu = True):
    """
    train: (np.array, np.array)
    val: (np.array, np.array)
    """
    X_train, Y_train = train

    if val:
        X_train, X_val = np.split(X_train,[54000])
        Y_train, Y_val = np.split(Y_train,[54000])
        n_val = len(X_val)
    n_train = len(X_train)
    optimizer = optimizers.NesterovAG(lr=init_lr)
    optimizer.setup(model)


    if cpu:
        model.to_cpu()
    update_start =perf_counter()
    for epoch in range(num_epochs):
        print('-'*20, 'epoch:', epoch+1, '-'*20)
        # training
        count = 0
        num_samples = 0
        train_loss = 0
        train_acc = 0
        train_start = perf_counter()
        print('training...')
        for t in range(0, n_train, batchsize):
            model.cleargrads()
            minibatch, labels = X_train[t:t+batchsize], Y_train[t:t+batchsize]
            if cpu:
                minibatch = cuda.to_cpu(minibatch)
                labels = cuda.to_cpu(labels)
            y = model(minibatch)
            loss = F.softmax_cross_entropy(y, labels)
            loss.backward()
            optimizer.update()
            y_pred = F.softmax(y)
            preds = cuda.to_cpu(y_pred.data).argmax(axis=1)
            labels = cuda.to_cpu(labels)
            train_loss += float(loss.data)*len(minibatch)
            train_acc += (labels==preds).sum()
            count += 1
            num_samples += len(minibatch)
        print('train_acc:', round(train_acc/num_samples, 4), 'train_loss:', round(train_loss/num_samples, 4))
        print('Took', perf_counter() - train_start, 'seconds.')

        index = np.random.permutation(np.arange(n_train))
        X_train = X_train[index]
        Y_train = Y_train[index]

        if val:
            # validation
            val_acc = 0
            val_loss = 0
            num_samples = 0
            count = 0
            model.train = False
            val_start = perf_counter()
            print('\n')
            print('validating...')
            for v in range(0, n_val, batchsize):
                minibatch, labels = X_val[v:v+batchsize], Y_val[v:v+batchsize]
                if cpu:
                    minibatch = cuda.to_cpu(minibatch)
                    labels = cuda.to_cpu(labels)
                y = model(minibatch)
                loss = F.softmax_cross_entropy(y, labels)
                y_pred = F.softmax(y)
                preds = cuda.to_cpu(y_pred.data).argmax(axis=1)
                val_loss += float(loss.data)*len(minibatch)
                labels = cuda.to_cpu(labels)
                val_acc += (labels==preds).sum()
                count+=1
                num_samples += len(minibatch)
            val_acc /= num_samples
            print('val_acc:', round(val_acc, 4), 'val_loss:', round(val_loss/num_samples, 4))
            print('Took', perf_counter()-val_start, 'seconds.')
            model.train = True
        serializers.save_npz(model_name+'.npz', model)

    print('\nTook',perf_counter()-update_start, 'seconds.')

#  テスト

In [ ]:
test_conv1 = result_c1_p[0,1,:6,:6] #ひとつめの畳み込み1後マップ
# test_n=np.array(test_conv1).reshape(6,6,6)
test_conv1

array([[-5.5282144e-03, -4.9126404e-03, -2.4221928e-03, -1.2354891e-02,
        -1.1151132e-02, -8.4754312e-03],
       [-2.8503714e-03, -7.3748855e-03, -4.9714718e-02, -1.2848492e-01,
        -3.2229477e-01, -5.5657709e-01],
       [-8.2547904e-04, -1.1642025e-02, -1.6760351e-01, -7.4102342e-01,
        -9.1166061e-01, -9.4134289e-01],
       [-2.4105157e-03, -5.5242665e-03, -2.2250561e-01, -5.1870048e-01,
        -7.6956898e-01, -8.2857323e-01],
       [-3.4598983e-03, -3.6797831e-03, -2.9009938e-02, -1.4555359e-01,
        -3.9911065e-01, -6.2640411e-01],
       [ 1.5062151e-03,  0.0000000e+00, -5.7461490e-03, -2.9071908e-02,
        -2.5416857e-01, -4.7376257e-01]], dtype=float32)

In [ ]:
test2 = conv2_param[0:1]
test2

array([[[[-7.04071894e-02,  8.75026286e-02, -9.96862277e-02,
           4.86757979e-03,  1.99564995e-04],
         [ 3.46713625e-02, -5.92313968e-02, -2.85311555e-03,
          -1.14815775e-02,  8.14119279e-02],
         [-6.49861619e-02,  5.93964756e-03, -2.13301964e-02,
          -1.05985068e-01,  2.18503669e-01],
         [-5.81500912e-03, -1.21385120e-01,  1.15013912e-01,
          -8.74129385e-02,  3.02814059e-02],
         [ 7.04499483e-02, -5.29442541e-02, -3.51820253e-02,
          -4.41128835e-02, -1.05622821e-02]],

        [[-1.32473394e-01, -1.00889310e-01, -1.14951646e-02,
           8.48300830e-02,  5.15816957e-02],
         [ 1.41206339e-01,  5.66691309e-02, -4.17335071e-02,
          -1.00301225e-02, -1.65780947e-01],
         [-7.84469619e-02, -8.33119899e-02,  2.20938604e-02,
           5.28950803e-02, -4.57542986e-02],
         [-4.80509549e-02, -1.26275748e-01, -1.04317283e-02,
           2.02628337e-02,  3.63978632e-02],
         [-6.39079288e-02,  1.62359506e-01, 

In [ ]:
data = result_c1_p[0,5]
data

In [ ]:
np.savetxt('out6.csv',result_c1_p[0,5],delimiter=',')

In [ ]:
test = conv2_param[0,5]
test

array([[ 0.00656932,  0.07385444, -0.15232979,  0.0061015 , -0.05129647],
       [ 0.02307912, -0.00385035,  0.05034337, -0.06839232,  0.15016612],
       [ 0.18908906, -0.01700627, -0.00122256,  0.02347898,  0.10321217],
       [ 0.15488131, -0.09845339, -0.0502234 , -0.08672468, -0.0908582 ],
       [-0.13384798,  0.02962278, -0.09463493,  0.12274083,  0.07416391]],
      dtype=float32)

In [ ]:
np.savetxt('param16.csv',conv2_param[0,5],delimiter=',')

# 事前準備

In [ ]:
# train_1とtrain_2を1つのフォルダにまとめる

import os
import shutil

# フォルダ作成
!mkdir train

# train_1からtrainへファイル移動
for filename in os.listdir('/content/train_1'):
  shutil.move(os.path.join('/content/train_1',filename),'/content/train')

# train_2からtrainへファイル移動
for filename in os.listdir('/content/train_2'):
  shutil.move(os.path.join('/content/train_2',filename),'/content/train')

# train_1とtrain_2のフォルダを削除
shutil.rmtree('/content/train_1')
shutil.rmtree('/content/train_2')


# フォルダ内のファイル数確認
import os

path_1 =  "/content/train"
files_1 = os.listdir(path_1)
num_1 = len(files_1)

path_2 =  "/content/test"
files_2 = os.listdir(path_2)
num_2 = len(files_2)

print('trainフォルダ画像数：',num_1)  # 60000になったらOK
print('test フォルダ画像数：',num_2)  # 10000になったらOK

trainフォルダ画像数： 60000
test フォルダ画像数： 10000
